In [ ]:
# hugging-face API login
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
%pip install langchain_community langchain_core langchain_huggingface
%pip install faiss-cpu
%pip install bitsandbytes

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings


# load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
import os


jd_lst = []
path = "../data/jd"
file_list = os.listdir(path)

# 각 JD별 항목들을 Dict로 변환하여 리스트로 저장
for f in file_list:
    with open(f"{path}{f}", "r") as fr:
        temp = {}
        for condition in fr.readlines():
            name, content = condition.split(" ", maxsplit=1)
            temp[name[:-1]] = content.strip()   # 콜론 제거

        # 모든 데이터에 직급 항목 추가
        if "직급" not in temp.keys():
            temp["직급"] = "명시 없음"
        jd_lst.append(temp)

In [ ]:
import pandas as pd


df = pd.read_excel("../data/rag.xlsx")
print(df.columns)
print(len(df.columns))

Index(['직무스트레스', '직무적합도', '직종', '직위', '근무형태', '입사연도', '고용상태', '산업분류',
       '월평균 임금(만원)', '평균근무시간', '고용보험가입', '산재보험가입', '신체강도요구', '물건들기요구',
       '허리굽힘요구', '시력요구', '집중력요구', '리더십요구', '컴퓨터기술요구', '직무난이도', '근무일정규칙성',
       '승진연공서열중시', '조기퇴직압박', '고령친화성', '학력', '성별', '나이', '결혼여부', '지역',
       '지난해 가구 총소득(만원)', '주관적건강', '전반적건강', '직무 만족도'],
      dtype='object')


33

In [ ]:
df = df[df['직무 만족도'] == '높음']    
df.drop('직무 만족도', axis=1, inplace=True)    # 직무 만족도가 '높음'으로 예측된 데이터만 필터링
df

,직무스트레스,직무적합도,직종,직위,근무형태,입사연도,고용상태,산업분류,월평균 임금(만원),평균근무시간,...,조기퇴직압박,고령친화성,학력,성별,나이,결혼여부,지역,지난해 가구 총소득(만원),주관적건강,전반적건강
1,그런 편이다,보통이다,단순노무 종사자,평직원,전일제 근로,2013,상용직 근로자,사업서비스업,140,50,...,전혀 그렇지 않다,전혀 그렇지 않다,고등학교 졸업,남자,57,미혼/이혼/별거/사별,서울,4986,좋은편,보통
3,그렇지 않은 편이다,보통이다,단순노무 종사자,평직원,전일제 근로,2014,상용직 근로자,건설업,180,40,...,전혀 그렇지 않다,전혀 그렇지 않다,고등학교 졸업,남자,60,미혼/이혼/별거/사별,서울,7200,보통,나쁜 편
4,그런 편이다,보통이다,단순노무 종사자,평직원,전일제 근로,2014,상용직 근로자,금융 및 부동산업,123,44,...,전혀 그렇지 않다,전혀 그렇지 않다,고등학교 졸업,여자,60,미혼/이혼/별거/사별,서울,6300,보통,보통
5,그런 편이다,보통이다,단순노무 종사자,직책 없음,전일제 근로,2013,일용직 근로자,사업서비스업,132,48,...,전혀 그렇지 않다,전혀 그렇지 않다,중학교 졸업,여자,57,미혼/이혼/별거/사별,서울,2500,좋은편,보통
6,그런 편이다,보통이다,단순노무 종사자,직책 없음,시간제 근로,2013,일용직 근로자,사업서비스업,100,40,...,전혀 그렇지 않다,전혀 그렇지 않다,고등학교 졸업,여자,67,미혼/이혼/별거/사별,서울,3800,보통,보통
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,그렇지 않은 편이다,보통이다,판매근로자,평직원,전일제 근로,2017,상용직 근로자,도매 및 소매업,180,48,...,그렇지 않은 편이다,그렇지 않은 편이다,대학교 졸업이상,여자,59,기혼,대전,5000,좋은편,좋은 편
1335,그런 편이다,보통이다,서비스근로자,직책 없음,전일제 근로,2021,임시직 근로자,공공행정업,110,25,...,전혀 그렇지 않다,전혀 그렇지 않다,대학교 졸업이상,여자,59,미혼/이혼/별거/사별,광주,4300,좋은편,보통
1336,그렇지 않은 편이다,보통이다,판매근로자,관리자급,전일제 근로,2022,상용직 근로자,도매 및 소매업,250,48,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,남자,60,미혼/이혼/별거/사별,광주,6000,좋은편,좋은 편
1338,그렇지 않은 편이다,보통이다,서비스근로자,직책 없음,시간제 근로,2021,상용직 근로자,보건 및 사회복지사업,30,35,...,그렇지 않은 편이다,전혀 그렇지 않다,대학교 졸업이상,여자,60,미혼/이혼/별거/사별,전북,1200,좋은편,좋은 편


In [ ]:
from typing import Dict


def process_jd(jd: Dict[str, str]) -> str:
    """JD 내 항목들을 필터링한 후 문자열로 변환

    Args:
        jd (Dict[str, str]): 개별 JD 파일

    Returns:
        str: 문자열로 변환된 JD
    """
    # 무관, 명시 없음 등이 포함된 항목들은 전부 제거
    exclude_tokens = ["명시 없음", "무관"]
    filtered_jd = {
        k: v.strip() for k, v in jd.items()
        if v and not any(token in v for token in exclude_tokens)
    }


    description = [
        f"{filtered_jd['기업명']}는 현재 {filtered_jd['담당업무']}를 담당할 {filtered_jd['근무형태']} 직원을 모집하고 있습니다.",
        f"근무지역은 {filtered_jd['근무지역']}이며, 소정근로시간은 {filtered_jd['소정근로시간']}, 근무요일은 {filtered_jd['근무요일']}입니다.",
        f"기업형태는 {filtered_jd['기업형태']}입니다." if '기업형태' in filtered_jd else "",
        f"복리후생으로는 {filtered_jd['복리후생']} 등이 제공됩니다." if '복리후생' in filtered_jd else "",
        f"직급은 {filtered_jd['직급']}입니다." if '직급' in filtered_jd else "",
        f"급여는 {filtered_jd['급여']}을 제공할 예정입니다." if '급여' in filtered_jd else "",
        f"학력은 {filtered_jd['학력']}을 우대합니다." if '학력' in filtered_jd else "",
        f"성별은 {filtered_jd['성별']}을 우대합니다." if '성별' in filtered_jd else "",
        f"나이는 {filtered_jd['나이']}을 우대합니다." if '나이' in filtered_jd else "",
        f"다음은 필요한 자격입니다: {filtered_jd['자격']}" if '자격' in filtered_jd else "",
        f"다음은 우대조건입니다: {filtered_jd['우대조건']}" if '우대조건' in filtered_jd else "",
    ]

    return " ".join(s for s in description if s)


In [ ]:
jd_lst

[{'title': '구미버스주식회사 시내버스 운전기사 모집',
  '기업명': '구미버스주식회사',
  '기업형태': '명시 없음',
  '업종': '명시 없음',
  '담당업무': '구미 시내버스 운전',
  '근무형태': '기간의 정함이 없는 근로계약',
  '급여': '연봉 53,000,000원 이상',
  '근무요일': '월 17일 만근',
  '소정근로시간': '주 52시간, 복격일 근무제 06:00-22:30',
  '근무지역': '경상북도 구미시',
  '복리후생': '퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험',
  '경력': '1년 이상 필수',
  '학력': '무관',
  '성별': '명시 없음',
  '나이': '명시 없음',
  '자격': '자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자',
  '우대조건': '명시 없음',
  '직급': '명시 없음'},
 {'title': '인사담당 중장년 경력직 채용',
  '기업명': '센텍주식회사',
  '기업형태': '중소기업',
  '업종': '기체 여과기 제조업',
  '담당업무': '인사채용 및 관리업무, 인사정책 및 절차 개발과 시행, 직원성과 평가 및 피드백 관리, 교육 및 개발 프로그램 기획과 운영, 노사관계 관리와 법적 준수 사항 점검, 급여 관리업무, 연말정산 관련업무, 퇴직연금 및 근태관리 업무, 사내 복리후생 제도 운영',
  '근무형태': '정규직',
  '급여': '명시 없음',
  '근무요일': '주 5일 월 화 수 목 금',
  '소정근로시간': '08:30-17:30',
  '근무지역': '서울특별시 서초구',
  '복리후생': '상여금, 퇴직금, 성과급, 4대보험, 연차제도',
  '경력': '인사관리 분야 5년 이상 근무',
  '학력': '4년제 이상 대학교 졸업',
  '성별': '명시 없음'

In [ ]:
import pandas as pd
from typing import Dict


def filter_education(df: pd.DataFrame, jd: Dict[str, str]) -> pd.DataFrame:
    """학력을 기준으로 필터링하는 함수

    Args:
        df (pd.DataFrame): 구직자 데이터
        jd (Dict[str, str]): 개별 JD dict

    Returns:
        pd.DataFrame: 필터링된 데이터프레임
    """
    # 학교 키워드 매핑
    keyword_score = {
        '초등학교': 1, '초등': 1, '초졸': 1,
        '중학교': 2, '중등': 2, '중졸': 2,
        '고교': 3, '고등학교': 3, '실업고': 3, '인문계고': 3, '고졸': 3,
        '대학교': 4, '대졸': 4, '대학': 4,
    }

    def get_score(text: str) -> int:
        if not isinstance(text, str):
            return 0
        for k, v in keyword_score.items():
            if k in text:
                return v
        return 0

    jd_score = get_score(jd.get("학력", ""))

    mask = df["학력"].map(get_score) >= jd_score
    return df.loc[mask].copy()


In [ ]:
def filter_gender(df: pd.DataFrame, jd: Dict[str, str]) -> pd.DataFrame:
    """성별 필터링하는 함수

    Args:
        df (pd.DataFrame): 구직자 데이터
        jd (Dict[str, str]): 개별 JD dict
    Returns:
        pd.DataFrame: 필터링된 데이터프레임
    """
    gender = {
        "여성": 1, "여자": 1, "남성": 2, "남자": 2
    }

    def get_gender(text):
        if not isinstance(text, str):
            return 0
        for k, v in gender.items():
            if k in text:
                return v
        return 0

    jd_score = get_gender(jd.get("성별", ""))

    if jd_score:
        mask = df["성별"].map(get_gender) == jd_score
        return df.loc[mask].copy()
    else:
        return df

In [ ]:
import re


def filter_age(df: pd.DataFrame, jd: Dict[str, str], tolerance: int= 3) -> pd.DataFrame:
    """나이에 따라 구직자를 필터링하는 함수

    Args:
        df (pd.DataFrame): 구직자 데이터
        jd (Dict[str, str]): 개별 JD dict
        tolerance (int, optional): 나이 범위값(±). Defaults to 3.

    Returns:
        pd.DataFrame: 필터링된 데이터
    """
    jd_age = re.search(r'\d+', jd.get("나이", ""))

    if jd_age:
        jd_age = int(jd_age.group())

        mask = (df["나이"] - jd_age).abs() <= tolerance
        return df.loc[mask].copy()
    else:
        return df.copy()

In [ ]:
from sentence_transformers import util

GROUP_NEARNESS = {
    "경북": ["경남", "울산", "부산"],
    "전남": ["전북", "광주"],
    "대전": ["충북", "충남"],
    "인천": ["서울", "경기"],
    "강원": ["충북", "경기"],
    "충남": ["충북", "대전"],
    "울산": ["경남", "부산", "경북"],
    "충북": ["충남", "대전"]
}


def filter_loc(df: pd.DataFrame, jd: Dict[str, str], embedding_model: HuggingFaceEmbeddings) -> pd.DataFrame:
    """거주 지역에 따라 필터링하는 함수

    Args:
        df (pd.DataFrame): 구직자 데이터
        jd (Dict[str, str]): 개별 JD dict
        embedding_model (HuggingFaceEmbeddings): 임베딩 모델

    Returns:
        pd.DataFrame: 필터링된 데이터프레임
    """
    loc_lst = list(set(df['지역']))
    jd_loc = jd['근무지역']
    cos_scores = [util.cos_sim(embedding_model.embed_query(loc), embedding_model.embed_query(jd_loc))[0][0] for loc in loc_lst]

    final_loc = loc_lst[cos_scores.index(max(cos_scores))]

    result = df[df['지역'] == final_loc]

    if len(result) < 50:
        # 50명 안 넘으면 그룹에서 하나씩 연결함.
        near_locs = GROUP_NEARNESS[final_loc]
        for near_loc in near_locs:
            result = pd.concat([result, df[df['지역'] == near_loc]], axis=0)
            if len(result) > 50:
                break

    return result

In [ ]:
# 실험
target_jd = jd_lst[0]

filtered_df = filter_loc(df, target_jd, embedding_model=embedding_model)

In [ ]:
target_jd

{'title': '구미버스주식회사 시내버스 운전기사 모집',
 '기업명': '구미버스주식회사',
 '기업형태': '명시 없음',
 '업종': '명시 없음',
 '담당업무': '구미 시내버스 운전',
 '근무형태': '기간의 정함이 없는 근로계약',
 '급여': '연봉 53,000,000원 이상',
 '근무요일': '월 17일 만근',
 '소정근로시간': '주 52시간, 복격일 근무제 06:00-22:30',
 '근무지역': '경상북도 구미시',
 '복리후생': '퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험',
 '경력': '1년 이상 필수',
 '학력': '무관',
 '성별': '명시 없음',
 '나이': '명시 없음',
 '자격': '자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자',
 '우대조건': '명시 없음',
 '직급': '명시 없음'}

In [ ]:
filtered_df

,직무스트레스,직무적합도,직종,직위,근무형태,입사연도,고용상태,산업분류,월평균 임금(만원),평균근무시간,...,조기퇴직압박,고령친화성,학력,성별,나이,결혼여부,지역,지난해 가구 총소득(만원),주관적건강,전반적건강
189,그렇지 않은 편이다,낮은 편이다,단순노무 종사자,평직원,전일제 근로,2012,임시직 근로자,사업서비스업,70,30,...,그렇지 않은 편이다,그렇지 않은 편이다,중학교 졸업,여자,55,미혼/이혼/별거/사별,경북,1000,좋은편,좋은 편
196,전혀 그렇지 않다,보통이다,단순노무 종사자,직책 없음,시간제 근로,2013,상용직 근로자,사업서비스업,70,5,...,그렇지 않은 편이다,그렇지 않은 편이다,초등학교 졸업이하,여자,71,미혼/이혼/별거/사별,경북,1005,보통,좋은 편
197,그렇지 않은 편이다,보통이다,단순노무 종사자,관리자급,전일제 근로,2018,상용직 근로자,사업서비스업,100,30,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,여자,67,기혼,경북,1476,좋은편,보통
338,매우 그렇다,보통이다,(준)전문가,직책 없음,시간제 근로,2016,임시직 근로자,사업서비스업,100,25,...,그렇지 않은 편이다,그렇지 않은 편이다,대학교 졸업이상,남자,58,미혼/이혼/별거/사별,경북,8000,매우좋음,보통
416,그렇지 않은 편이다,보통이다,"장치, 기계조작 및 조립 종사자",평직원,전일제 근로,2016,임시직 근로자,사업서비스업,150,40,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,남자,62,미혼/이혼/별거/사별,경북,1800,보통,나쁜 편
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,그렇지 않은 편이다,낮은 편이다,단순노무 종사자,직책 없음,시간제 근로,2021,임시직 근로자,공공행정업,27,9,...,전혀 그렇지 않다,전혀 그렇지 않다,초등학교 졸업이하,여자,67,미혼/이혼/별거/사별,경남,1200,보통,보통
1271,그렇지 않은 편이다,보통이다,단순노무 종사자,직책 없음,시간제 근로,2022,임시직 근로자,문화/기타 서비스업,24,15,...,그렇지 않은 편이다,그렇지 않은 편이다,초등학교 졸업이하,여자,77,기혼,경남,660,좋은편,보통
1305,그런 편이다,보통이다,단순노무 종사자,직책 없음,전일제 근로,2017,상용직 근로자,생산 및 제조업,200,48,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,여자,60,미혼/이혼/별거/사별,경남,3000,좋은편,좋은 편
1307,그렇지 않은 편이다,보통이다,서비스근로자,직책 없음,시간제 근로,2022,임시직 근로자,보건 및 사회복지사업,100,40,...,그렇지 않은 편이다,그렇지 않은 편이다,대학교 졸업이상,여자,60,기혼,경남,1200,좋은편,좋은 편


In [ ]:
filtered_df = filter_age(filtered_df, target_jd)
filtered_df = filter_gender(filtered_df, target_jd)
filtered_df = filter_education(filtered_df, target_jd)

In [ ]:
filtered_df

,직무스트레스,직무적합도,직종,직위,근무형태,입사연도,고용상태,산업분류,월평균 임금(만원),평균근무시간,...,조기퇴직압박,고령친화성,학력,성별,나이,결혼여부,지역,지난해 가구 총소득(만원),주관적건강,전반적건강
189,그렇지 않은 편이다,낮은 편이다,단순노무 종사자,평직원,전일제 근로,2012,임시직 근로자,사업서비스업,70,30,...,그렇지 않은 편이다,그렇지 않은 편이다,중학교 졸업,여자,55,미혼/이혼/별거/사별,경북,1000,좋은편,좋은 편
196,전혀 그렇지 않다,보통이다,단순노무 종사자,직책 없음,시간제 근로,2013,상용직 근로자,사업서비스업,70,5,...,그렇지 않은 편이다,그렇지 않은 편이다,초등학교 졸업이하,여자,71,미혼/이혼/별거/사별,경북,1005,보통,좋은 편
197,그렇지 않은 편이다,보통이다,단순노무 종사자,관리자급,전일제 근로,2018,상용직 근로자,사업서비스업,100,30,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,여자,67,기혼,경북,1476,좋은편,보통
338,매우 그렇다,보통이다,(준)전문가,직책 없음,시간제 근로,2016,임시직 근로자,사업서비스업,100,25,...,그렇지 않은 편이다,그렇지 않은 편이다,대학교 졸업이상,남자,58,미혼/이혼/별거/사별,경북,8000,매우좋음,보통
416,그렇지 않은 편이다,보통이다,"장치, 기계조작 및 조립 종사자",평직원,전일제 근로,2016,임시직 근로자,사업서비스업,150,40,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,남자,62,미혼/이혼/별거/사별,경북,1800,보통,나쁜 편
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,그렇지 않은 편이다,낮은 편이다,단순노무 종사자,직책 없음,시간제 근로,2021,임시직 근로자,공공행정업,27,9,...,전혀 그렇지 않다,전혀 그렇지 않다,초등학교 졸업이하,여자,67,미혼/이혼/별거/사별,경남,1200,보통,보통
1271,그렇지 않은 편이다,보통이다,단순노무 종사자,직책 없음,시간제 근로,2022,임시직 근로자,문화/기타 서비스업,24,15,...,그렇지 않은 편이다,그렇지 않은 편이다,초등학교 졸업이하,여자,77,기혼,경남,660,좋은편,보통
1305,그런 편이다,보통이다,단순노무 종사자,직책 없음,전일제 근로,2017,상용직 근로자,생산 및 제조업,200,48,...,그렇지 않은 편이다,그렇지 않은 편이다,고등학교 졸업,여자,60,미혼/이혼/별거/사별,경남,3000,좋은편,좋은 편
1307,그렇지 않은 편이다,보통이다,서비스근로자,직책 없음,시간제 근로,2022,임시직 근로자,보건 및 사회복지사업,100,40,...,그렇지 않은 편이다,그렇지 않은 편이다,대학교 졸업이상,여자,60,기혼,경남,1200,좋은편,좋은 편


In [ ]:
# 특정 JD로 실험
target_jd_str = process_jd(target_jd)
target_jd_str

'구미버스주식회사는 현재 구미 시내버스 운전를 담당할 기간의 정함이 없는 근로계약 직원을 모집하고 있습니다. 근무지역은 경상북도 구미시이며, 소정근로시간은 주 52시간, 복격일 근무제 06:00-22:30, 근무요일은 월 17일 만근입니다. 복리후생으로는 퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험 등이 제공됩니다. 급여는 연봉 53,000,000원 이상을 제공할 예정입니다. 다음은 필요한 자격입니다: 자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자'

In [ ]:
from typing import Optional


def as_str(x) -> Optional[str]:
    if pd.isna(x):
        return None
    else:
        s = str(x).strip()
        return None if s and s.lower() in ['nan', 'none'] else s

def make_prev_job_string(row: pd.Series) -> str:
    """구직자 데이터 내 전직장 정보를 문자열로 바꾸는 함수

    Args:
        row (pd.Series): 개별 구직자 데이터

    Returns:
        _str_: 구직자 전직장 정보 문자열
    """
    ind = as_str(row.get("산업분류"))
    job = as_str(row.get("직종"))
    position = as_str(row.get("직위"))
    work_form = as_str(row.get("근무형태"))
    hire_form = as_str(row.get("고용상태"))
    wage = as_str(row.get("월평균 임금(만원)"))
    work_time = as_str(row.get("평균근무시간"))

    parts = []
    if ind and job and position:
        parts.append(f"{ind} 산업의 {job} 및 {position}로 근무")
    if work_form or hire_form:
        parts.append("·".join([p for p in [work_form, hire_form] if p]))
    line1 = ", ".join(parts) if parts else ""

    parts2 = []
    if wage:
        parts2.append(f"월 평균 임금 {wage}만원")
    if work_time:
        parts2.append(f"월 평균 근무시간 {work_time}시간")
    line2 = ", ".join(parts2) if parts2 else ""

    return f"최근 직장 경력 요약: {line1}. {line2}" if line1 or line2 else ""

In [ ]:
# document화
from langchain_core.documents import Document

docs = []
for i, row in filtered_df.iterrows():
  docs.append(Document(page_content=make_prev_job_string(row),  metadata={"idx": i, "location": row['지역'], "gender": row['성별'], "age": row['나이'], "edu": row['학력']}))
docs

[Document(metadata={'idx': 189, 'location': '경북', 'gender': '여자', 'age': 55, 'edu': '중학교 졸업'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 단순노무 종사자 및 평직원로 근무, 전일제 근로·임시직 근로자. 월 평균 임금 70만원, 월 평균 근무시간 30시간'),
 Document(metadata={'idx': 196, 'location': '경북', 'gender': '여자', 'age': 71, 'edu': '초등학교 졸업이하'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 단순노무 종사자 및 직책 없음로 근무, 시간제 근로·상용직 근로자. 월 평균 임금 70만원, 월 평균 근무시간 5시간'),
 Document(metadata={'idx': 197, 'location': '경북', 'gender': '여자', 'age': 67, 'edu': '고등학교 졸업'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 단순노무 종사자 및 관리자급로 근무, 전일제 근로·상용직 근로자. 월 평균 임금 100만원, 월 평균 근무시간 30시간'),
 Document(metadata={'idx': 338, 'location': '경북', 'gender': '남자', 'age': 58, 'edu': '대학교 졸업이상'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 (준)전문가 및 직책 없음로 근무, 시간제 근로·임시직 근로자. 월 평균 임금 100만원, 월 평균 근무시간 25시간'),
 Document(metadata={'idx': 416, 'location': '경북', 'gender': '남자', 'age': 62, 'edu': '고등학교 졸업'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 장치, 기계조작 및 조립 종사자 및 평직원로 근무, 전일제 근로·임시직 근로자. 월 평균 임금

In [ ]:
from langchain_community.vectorstores import FAISS

# FAISS 모델을 통해 코사인 유사도 계산
vectorstore = FAISS.from_documents(docs, embedding_model)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
print(target_jd)

{'title': '구미버스주식회사 시내버스 운전기사 모집', '기업명': '구미버스주식회사', '기업형태': '명시 없음', '업종': '명시 없음', '담당업무': '구미 시내버스 운전', '근무형태': '기간의 정함이 없는 근로계약', '급여': '연봉 53,000,000원 이상', '근무요일': '월 17일 만근', '소정근로시간': '주 52시간, 복격일 근무제 06:00-22:30', '근무지역': '경상북도 구미시', '복리후생': '퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험', '경력': '1년 이상 필수', '학력': '무관', '성별': '명시 없음', '나이': '명시 없음', '자격': '자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자', '우대조건': '명시 없음', '직급': '명시 없음'}


In [ ]:
retriever.invoke(target_jd_str)

/tmp/ipython-input-3398154845.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(target_jd_str)


[Document(id='861b6ad3-7382-4f67-ac32-67bc65b1a20a', metadata={'idx': 468, 'location': '경남', 'gender': '남자', 'age': 57, 'edu': '고등학교 졸업'}, page_content='최근 직장 경력 요약: 운수업 산업의 서비스근로자 및 직책 없음로 근무, 전일제 근로·상용직 근로자. 월 평균 임금 130만원, 월 평균 근무시간 48시간'),
 Document(id='3d391e41-08ae-44ab-b118-6fa17913c8a4', metadata={'idx': 552, 'location': '경북', 'gender': '여자', 'age': 67, 'edu': '초등학교 졸업이하'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 서비스근로자 및 직책 없음로 근무, 시간제 근로·임시직 근로자. 월 평균 임금 50만원, 월 평균 근무시간 25시간'),
 Document(id='821f6eb6-b383-4d91-9b60-ac991d4c39be', metadata={'idx': 924, 'location': '경남', 'gender': '여자', 'age': 64, 'edu': '중학교 졸업'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 단순노무 종사자 및 직책 없음로 근무, 시간제 근로·임시직 근로자. 월 평균 임금 50만원, 월 평균 근무시간 16시간'),
 Document(id='b5e78e17-4e73-4bf2-8d05-f7ca118fc9f3', metadata={'idx': 912, 'location': '경남', 'gender': '여자', 'age': 61, 'edu': '고등학교 졸업'}, page_content='최근 직장 경력 요약: 사업서비스업 산업의 서비스근로자 및 직책 없음로 근무, 시간제 근로·임시직 근로자. 월 평균 임금 60만원, 월 평균 근무시간 15시간'),
 Document(id='6bfc3

## Langchain

In [ ]:
# load llm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline


MODEL_ID = "google/gemma-3n-E4B-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
gen_pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.05,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=gen_pipe)

Device set to use cuda:0


In [ ]:
import pandas as pd
from langchain_core.documents import Document


def doc2json(doc: Document, df: pd.DataFrame) -> Dict[str, str]:
    idx = int(doc.metadata['idx'])
    lookup_row = df.loc[idx]
    result = {"idx": idx}
    result.update(lookup_row.to_dict())  # 모든 컬럼을 dict로 변환 후 병합
    return result

In [ ]:
from typing import List, Annotated
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


def make_parser(n: int) -> JsonOutputParser:
    class SelectedCandidate(BaseModel):
        idx: int = Field(..., description="지원자 고유 ID")
        reason: str = Field(..., description="추천 이유 한 문장")

    class SelectionResult(BaseModel):
        selected: Annotated[
            List[SelectedCandidate],
            Field(min_length=n, max_length=n)
        ]

    return JsonOutputParser(pydantic_object=SelectionResult)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


# 프롬프트 구성
prompt_template = ChatPromptTemplate.from_messages([
    ("system",
    "너는 헤드헌터 보조 에이전트다. JD와 후보자 정보를 비교해 정확히 {n}명을 고르되, "
    "출력은 하나의 JSON만 허용한다: {{\"selected\": [{{\"idx\": <int>, \"reason\": <str>}}, ...]}}."
    " 조건: "
    "(1) idx 중복 금지, "
    "(2) 거짓·추측 표현 금지(예: '~예상됩니다', '~보입니다', '~가능성'), "
    "(3) 각 reason은 2문장 이상이며, **후보별 문장·표현·근거 조합이 중복되지 않게 할 것**, "
    "(4) 각 reason에는 JD와 후보자 모두에 **정확히 명시된 값**을 최소 3개 이상 포함하고, "
    "그 중 적어도 1개는 다른 후보들과 겹치지 않는 **고유 필드**여야 함 "
    "(예: 입사연도, 보험가입, 평균근무시간, 직무스트레스/난이도/적합도, 승진연공서열중시, 조기퇴직압박, 고령친화성, 신체·인지 요구 등), "
    "(5) reason에는 **idx·번호·후보** 등 식별자 언급 금지, "
    "(6) **이유 중심 인과형 서술**로 작성: 첫 문장에서 'JD 조건과 후보 특성이 어떻게 맞물리는지'를 설명하고, "
    "두 번째 이후 문장에서 '이 특성들이 직무 수행에 어떤 긍정적 결과를 줄지'를 논리적으로 연결할 것, "
    "(7) 서술 구조는 후보마다 달리 하고, 같은 형식 반복 금지."
    ),
    ("human",
     "구직 공고(JD):\n{job_posting}\n\n"
     "후보자 목록(JSON):\n{candidates_json}\n\n"
     "규칙:\n"
     "1) candidates_json의 idx에서만 선택\n"
     "2) 정확히 {n}명 선택, idx/문장/근거 조합 중복 금지\n"
     "3) reason에는 JD·후보자에 모두 있는 구체 값만 사용\n"
     "{format_instructions}")
])

In [ ]:
import pandas as pd
from operator import itemgetter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables.base import RunnableSerializable


def build_chain(docs: Document, embedding_model: HuggingFaceEmbeddings, llm: HuggingFacePipeline, prompt: ChatPromptTemplate, df: pd.DataFrame, n: int) -> RunnableSerializable:
    """chain 구성 함수

    Args:
        docs (Document): 벡터화된 문서(전 직장 정보)
        embedding_model (HuggingFaceEmbeddings): 임베딩 모델
        llm (HuggingFacePipeline): Generator
        prompt (ChatPromptTemplate): 프롬프트
        df (pd.DataFrame): 원본 구직자 데이터(병합 시 필요)
        n (int): 최종 후보군 수

    Returns:
        RunnableSerializable: chain
    """
    retriever = FAISS.from_documents(docs, embedding_model).as_retriever(search_kwargs={"k": 2 * n})    # 최종 후보군의 2배만큼 retriever로 먼저 뽑음.
    parser = make_parser(n)
    fmt = parser.get_format_instructions()

    def get_topn_candidates(x):
        docs = retriever.invoke(x["query"])
        return [doc2json(d, df) for d in docs]

    chain = (
        {
            "job_posting": itemgetter("job_posting"),
            "candidates_json": lambda x: get_topn_candidates(x)
        }
        | prompt.partial(format_instructions=fmt, n=n)
        | llm
        | parser
    )
    return chain

In [ ]:
chain = build_chain(docs, embedding_model, llm, prompt_template, df, 3)

In [ ]:
target_jd_str

'구미버스주식회사는 현재 구미 시내버스 운전를 담당할 기간의 정함이 없는 근로계약 직원을 모집하고 있습니다. 근무지역은 경상북도 구미시이며, 소정근로시간은 주 52시간, 복격일 근무제 06:00-22:30, 근무요일은 월 17일 만근입니다. 복리후생으로는 퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험 등이 제공됩니다. 급여는 연봉 53,000,000원 이상을 제공할 예정입니다. 다음은 필요한 자격입니다: 자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자'

In [ ]:
payload = {
    "query": target_jd_str,
    "job_posting": target_jd
}

chain.invoke(payload)

{'selected': [{'idx': 924,
   'reason': '이 지원자는 사업서비스업 분야에서 시간제 근로자로 근무하며, 월평균 임금이 50만원으로, 이는 구미버스주식회사의 연봉 53,000,000원 이상 조건을 충족하지는 않지만, 안정적인 근무 형태를 선호하는 점과 전반적 건강 상태가 좋다는 점은 장기적인 직무 수행에 도움이 될 수 있다.'},
  {'idx': 912,
   'reason': '이 지원자는 사업서비스업 분야에서 시간제 근로자로 근무하며, 월평균 임금이 60만원으로, 이는 구미버스주식회사의 연봉 53,000,000원 이상 조건을 충족하지는 않지만, 근무일정이 규칙적이고 주관적 건강 상태가 매우 좋다는 점은 업무 집중력을 높여 시내버스 운전 업무에 긍정적인 영향을 미칠 수 있다.'},
  {'idx': 684,
   'reason': '이 지원자는 단순노무 종사자로서 시간제 근로자로 근무하며, 월평균 임금이 50만원으로, 이는 구미버스주식회사의 연봉 53,000,000원 이상 조건을 충족하지는 않지만, 승진연공서열중시를 중요하게 생각하지 않고, 업무 난이도가 높지 않다는 점은 장기적인 직무 만족도를 높이는 데 기여할 수 있다.'}]}

In [ ]:
jd_pair = [(process_jd(jd), jd) for jd in jd_lst]
jd_pair

[('구미버스주식회사는 현재 구미 시내버스 운전를 담당할 기간의 정함이 없는 근로계약 직원을 모집하고 있습니다. 근무지역은 경상북도 구미시이며, 소정근로시간은 주 52시간, 복격일 근무제 06:00-22:30, 근무요일은 월 17일 만근입니다. 복리후생으로는 퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험 등이 제공됩니다. 급여는 연봉 53,000,000원 이상을 제공할 예정입니다. 다음은 필요한 자격입니다: 자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자',
  {'title': '구미버스주식회사 시내버스 운전기사 모집',
   '기업명': '구미버스주식회사',
   '기업형태': '명시 없음',
   '업종': '명시 없음',
   '담당업무': '구미 시내버스 운전',
   '근무형태': '기간의 정함이 없는 근로계약',
   '급여': '연봉 53,000,000원 이상',
   '근무요일': '월 17일 만근',
   '소정근로시간': '주 52시간, 복격일 근무제 06:00-22:30',
   '근무지역': '경상북도 구미시',
   '복리후생': '퇴직연금 가입, 목욕권 지급, 장갑 지급, 신문 지급, 유니폼 지급, 법정공휴일 유급수당, 추석 설 근로자의 날 선물 지급, 대학생 자녀 학자금, 국민연금, 고용보험, 산재보험, 건강보험',
   '경력': '1년 이상 필수',
   '학력': '무관',
   '성별': '명시 없음',
   '나이': '명시 없음',
   '자격': '자동차운전면허 1종대형, 버스운전자격증 필수, 버스운전자 양성교육 이수자',
   '우대조건': '명시 없음',
   '직급': '명시 없음'}),
 ('센텍주식회사는 현재 인사채용 및 관리업무, 인사정책 및 절차 개발과 시행, 직원성과 평가 및 피드백 관리, 교육 및 개발 프로그램 기획과 운영, 노사관계 관리와

In [ ]:
from tqdm import tqdm


result = []
for jd_str, jd in tqdm(jd_pair):
  payload = {
    "query": jd_str,
    "job_posting": jd
  }
  result.append(chain.invoke(payload))

result

100%|██████████| 16/16 [18:47<00:00, 70.44s/it]


[{'selected': [{'idx': 912,
    'reason': "JD에서 요구하는 '버스운전면허 1종대형'과 후보자의 '고용상태'가 '임시직 근로자'이지만, 사업서비스업에서 15시간 근무하는 시간제 근로자로서 '근무일정규칙성'이 '예'이고, '승진연공서열중시'가 '전혀 그렇지 않다'는 점은 직무 스트레스가 낮다는 점에서 장점입니다."},
   {'idx': 924,
    'reason': "JD에서 필수 조건인 '버스운전자격증'을 보유한 후보자이며, '고용보험가입' 상태가 '미가입'이지만 '산재보험가입'되어 있어 안전 관리에 대한 인식이 높습니다. 또한 '신체강도요구'가 '그런 편이다'라는 점은 운전 업무에 필요한 신체적 능력을 갖추고 있음을 시사합니다."},
   {'idx': 684,
    'reason': "JD에서 요구하는 '자동차운전면허 1종대형'을 보유한 후보자이며, '고용보험가입' 상태가 '미가입'이지만 '산재보험가입'되어 있어 안전 관리에 대한 인식이 높습니다. 또한 '직무난이도'가 '그렇지 않은 편이다'라는 점은 운전 업무에 대한 부담감이 적다는 것을 의미합니다."}]},
 {'selected': [{'idx': 1258,
    'reason': 'JD에서 요구하는 높은 집중력 요구사항과 후보자의 매우 그렇다는 응답은 직무 난이도에 대한 높은 적응력을 시사합니다. 또한, JD에서 강조하는 컴퓨터 기술 요구사항과 후보자의 매우 그렇다는 응답은 업무 효율성 향상에 기여할 것으로 예상됩니다.'},
   {'idx': 1231,
    'reason': 'JD에서 제시된 인사채용 및 관리업무는 후보자의 서비스근로자 직종과 관리자급 직위 경험을 바탕으로 수행 가능합니다. 특히, JD에서 요구하는 우수한 대인관계 능력과 후보자의 그런 편이라는 답변은 원활한 커뮤니케이션을 통해 팀워크를 강화할 수 있을 것으로 기대됩니다.'},
   {'idx': 474,
    'reason': 'JD에서 요구하는 인사정책 및 절차 개발과 시행 업무는 후

In [ ]:
import time

st = time.time()
sample_jd_str, sample_jd = jd_pair[0]
sample_payload = {
  "query": sample_jd_str,
  "job_posting": sample_jd
}

sample_result = chain.invoke(sample_payload)
ed = time.time()

print(sample_result)
print(f"time: {ed - st}")

{'selected': [{'idx': 912, 'reason': '이 지원자는 시간제 근로 형태로 경남 지역에서 근무하며, 지난 해 가구 총소득이 4800만원으로 안정적인 경제 기반을 갖추고 있습니다.  JD에서 요구하는 1년 이상의 경력 조건은 충족하지 않지만, 사업서비스업 분야에서 근무한 경험이 있어 업무 적응력이 높을 것으로 기대됩니다.'}, {'idx': 924, 'reason': '이 지원자는 시간제 근로 형태로 경남 지역에서 근무하며, 지난 해 가구 총소득이 932만원으로 안정적인 경제 기반을 갖추고 있습니다.  JD에서 요구하는 1년 이상의 경력 조건은 충족하지 않지만, 사업서비스업 분야에서 근무한 경험이 있어 업무 적응력이 높을 것으로 기대됩니다.'}, {'idx': 684, 'reason': '이 지원자는 시간제 근로 형태로 경북 지역에서 근무하며, 지난 해 가구 총소득이 2400만원으로 안정적인 경제 기반을 갖추고 있습니다.  JD에서 요구하는 1년 이상의 경력 조건은 충족하지 않지만, 단순노무 종사자로 업무 경험이 있어 기본적인 운전 업무에 대한 이해도가 있을 것으로 예상됩니다.'}]}
time: 68.2486662864685
